In [1]:
import numpy as np
import cv2
import sklearn.preprocessing as preprocessing
from skimage.feature import haar_like_feature, haar_like_feature_coord, draw_haar_like_feature
from skimage.transform.integral import integral_image
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from time import time

In [2]:
def leerDataset(name,numIm):
    '''
    Función encargada de leer un dataset
    '''
    image=cv2.imread(name+'0.png',0)
    fil,col = image.shape
    caract=image.shape[0]*image.shape[1]
    matriz=np.zeros((numIm,caract))
    matriz[0,:]=image.ravel()
    for i in range(1,numIm):
        image=cv2.imread(name+str(i)+'.png',0)
        matriz[i,:]=image.ravel()
    return matriz,fil,col

In [3]:
def normalizarImagen(matriz):
    '''
    Función encargada de normalizar todas las imagenes de una matriz
    '''
    matriz = preprocessing.scale(matriz,axis=1)
    return matriz

In [4]:
def ampliarDataset(matriz,fil_im,col_im):
    '''
    Esta función hace un flip de todas la imagenes de la matriz
    '''
    fil,col = matriz.shape
    matrizAmpliada = np.zeros((fil*2,col))
    matrizAmpliada[0:fil,:] = matriz
    for i in range(fil):
        imagen = matriz[i,:].reshape(fil_im,col_im)
        matrizAmpliada[fil + i, :] = imagen[:,::-1].ravel()
    return matrizAmpliada

In [5]:
def preprocesamiento(opc):
    '''
    Función encargada de preprocesar las images
    '''
    if (opc==1):
        nameCaras = 'dataset1/caras/'
        matrizCaras,filCaras,colCaras = leerDataset(nameCaras,100)
        
        nameNoCaras ='dataset1/nocaras/'
        matrizNoCaras,filNoCaras,colNoCaras = leerDataset(nameNoCaras,100)
    else:
        nameCaras = 'dataset2/face/'
        matrizCaras,filCaras,colCaras = leerDataset(nameCaras,2428)
        
        nameNoCaras ='dataset2/non-face/'
        matrizNoCaras,filNoCaras,colNoCaras = leerDataset(nameNoCaras,4547)

    matrizCaras = normalizarImagen(matrizCaras)
    matrizNoCaras = normalizarImagen(matrizNoCaras)
    
    matrizCarasAmpliada = ampliarDataset(matrizCaras,filCaras, colCaras)
    matrizNoCarasAmpliada = ampliarDataset(matrizNoCaras,filNoCaras,colNoCaras)
   
    return matrizCarasAmpliada,matrizNoCarasAmpliada,filCaras,colCaras

In [6]:
def calcular_carHar(setImagenes,filImagenes,colImagenes,coordenadas=None,typeCord = None):
    '''
    Función encargada de calcular las caracteristicas Haar de una imagen
    '''
    numImagenes,pixeles = setImagenes.shape
        
    imagen = integral_image(setImagenes[0,:].reshape(filImagenes,colImagenes))
    caracteristics = haar_like_feature(imagen, r=0, c=0, width=colImagenes, height=filImagenes, feature_type = typeCord, feature_coord=coordenadas)
    caractImagenes = np.zeros((numImagenes,len(caracteristics)))
    caractImagenes[0,:] = caracteristics
    
    for i in range(1,numImagenes):
        imagen = integral_image(setImagenes[i,:].reshape(filImagenes,colImagenes))
        caractImagenes[i,:] = haar_like_feature(imagen, r=0, c=0, width=colImagenes, height=filImagenes, feature_type = typeCord,  feature_coord=coordenadas)
        
    return caractImagenes

In [7]:
def entrenamiento(caracteristicasCaras,caracteristicasNoCaras,n_caract):
    '''
    Función encargada de entrenar un Ada Boost utilizando n_caract caracteristicas
    clase 0 = NoCara
    clase 1 = Cara
    '''
    numCaras,caractCaras = caracteristicasCaras.shape
    numNoCaras,caractNoCaras = caracteristicasNoCaras.shape
    
    yCaras = np.ones(numCaras).reshape(-1,1)
    yNoCaras = np.zeros(numNoCaras).reshape(-1,1)
    
    X = np.vstack( (caracteristicasNoCaras, caracteristicasCaras) )
    y = np.vstack( (yNoCaras, yCaras) ).ravel()

    X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.25,random_state=42)
    Ada = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(criterion='entropy',max_depth=1), n_estimators=n_caract, random_state=42)
    Ada.fit(X_train,y_train)
    accVal = Ada.score(X_val,y_val)
    
    return Ada,accVal

In [8]:
def intersection_over_union(boxA, boxB):
    # determinamos las coordenadas de las intersecciones
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # computamos el area de la interseccion
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # compute el area de los dos rectangulos
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # Aplicamos la formula del intersecion_over_union
    iou = interArea / float(boxAArea + boxBArea - interArea)

    return iou

In [9]:
#Obtención de las coordenadas de haar y sus respectivos tipos y aplicarlas sobre la imagen
def obtenerCaracteristicas(matrizCaras,matrizNoCaras,filImagen,colImagen):
    '''
    Función encargada de obtener las caracteristicas, coordenadas y tipos de las imagenes de entrenamiento
    '''
    cord,typecord = haar_like_feature_coord(filImagen,colImagen)
    
    caracteristicasCaras = calcular_carHar(matrizCaras,filImagen,colImagen,cord,typecord)
    caracteristicasNoCaras = calcular_carHar(matrizNoCaras,filImagen,colImagen,cord,typecord)
    return cord,typecord,caracteristicasCaras,caracteristicasNoCaras

In [10]:
#Entrenamiento con todas las caracteristicas
#acc15=0.96, acc25=0.98, acc30=0.98, acc40=0.98
def entrenarClasificadores(caracteristicasCaras,caracteristicasNoCaras,matrizCaras,matrizNoCaras,cord,typecord,n_caract):
    
    '''
    Esta función entrena 5 clasificadores de cara/noCaras para poder aplicar la detección de caras en cascada
    '''
    caract_clasif = np.array(np.linspace(5,n_caract,5,dtype=int)) # 5 clasificadores con entre 5 y n_caract caractericticas
    Clasificadores = np.zeros(shape=caract_clasif.shape[0],dtype=object)
    
    Ada,accVal = entrenamiento(caracteristicasCaras,caracteristicasNoCaras, n_caract)
    
    #Creación de las cordenadas,tipos y aplicación en ejemplos de las caracteristicas importantes
    cord_validas = np.argwhere(Ada.feature_importances_!=0)
    cord_validas_cascada = np.zeros(shape=(caract_clasif.shape[0],cord_validas.shape[0]))
    
    carac_val = cord[cord_validas]
    t_carac_val = typecord[cord_validas]
    
    caracCarasImport = calcular_carHar(matrizCaras,filImagen,colImagen,carac_val,t_carac_val)
    caracNoCarasImport = calcular_carHar(matrizNoCaras,filImagen,colImagen,carac_val,t_carac_val)
    Ada,accVal = entrenamiento(caracCarasImport,caracNoCarasImport,n_caract)
    print 'Precisión del clasificador con {} caracteristicas: {}'.format(n_caract, accVal)
    Clasificadores[4] = Ada
    cord_validas_cascada[4,:] = (Ada.feature_importances_!=0)
    #Crear los clasificadores con diferente numero de caracteristicas importantes 
    #y entrenarlos para luego poder predecir en cascada
    for i in range(caract_clasif.shape[0]-1):
        n_c = caract_clasif[i]
        #Entrenamiento del clasificador con n_c caracteristicas
        Ada,accVal = entrenamiento(caracCarasImport,caracNoCarasImport,n_c)
        cord_validas_cascada[i,:] = (Ada.feature_importances_!=0)
        
        caracCarasImport_nc = calcular_carHar(matrizCaras,filImagen,colImagen,carac_val[(Ada.feature_importances_!=0)],t_carac_val[(Ada.feature_importances_!=0)])      
        caracNoCarasImport_nc = calcular_carHar(matrizNoCaras,filImagen,colImagen,carac_val[(Ada.feature_importances_!=0)],t_carac_val[(Ada.feature_importances_!=0)])
        Ada,accVal = entrenamiento(caracCarasImport_nc,caracNoCarasImport_nc,n_c)
        print 'Precisión del clasificador con {} caracteristicas: {}'.format(n_c, accVal)
        Clasificadores[i] = Ada
    return Clasificadores,cord_validas_cascada,carac_val,t_carac_val

In [11]:
#Codigo para ampliar, reducir imagenes
#height, width = src.shape[:2]
#dst = cv2.resize(src, (2*width, 2*height), interpolation = cv2.INTER_LINEAR)#Ampliar
#dst = cv2.resize(src, (width/2, height/2), interpolation = cv2.INTER_AREA)#Reducir

#predict_proba(X)# Devuelve la prediccion en probabilidad para luego poder hacer intersection over union
#https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/
#https://docs.opencv.org/3.1.0/d7/d8b/tutorial_py_face_detection.html
#https://docs.opencv.org/2.4/modules/core/doc/drawing_functions.html

In [12]:
def eliminar_caras_reduntantes(prob_cara,fil,col):
    '''
    Esta función suprime las caras redundantes mediante la supresión de no mínimos.
    '''
    sin_ver = np.copy(prob_cara)
    m=1
    while ((sin_ver[:,:,0] == 1).any()):
        ind_max = np.unravel_index(np.argmax(sin_ver[:,:,1]), sin_ver.shape[:2])
        m+=1
        sin_ver[ind_max[0],ind_max[1],0] = 0
        sin_ver[ind_max[0],ind_max[1],1] = 0
        #Box = esquina izda superior x, esquina izda superior y, esquina derecha inferior x, esquina derecha inferior y
        boxA = np.zeros(shape=(4))
        boxB = np.zeros(shape=(4))
        boxA[0] = ind_max[0]
        boxA[1] = ind_max[1]
        boxA[2] = ind_max[0] + 23
        boxA[3] = ind_max[1] + 23
        for i in range(ind_max[0] - 24, ind_max[0] + 24):
            for j in range(ind_max[1] - 24, ind_max[1] + 24):
                if (((ind_max[0] !=i) or (ind_max[1] != j)) and (i >=0) and (j>=0) and (i <= fil - 24) and (j<=col-24) and (prob_cara[i,j,0]==1)):
                    boxB[0] = i
                    boxB[1] = j
                    boxB[2] = i + 23
                    boxB[3] = j + 23
                    if (intersection_over_union(boxA, boxB) > 0.5):
                        prob_cara[i,j,0] = 0
                        sin_ver[i,j,0] = 0
                        sin_ver[i,j,1] = 0
    return prob_cara

In [13]:
def detectar_caras(imagen,Clasificadores,cord_validas,tipo_caracteristicas_validas,caracteristicas_validas):
    '''
    Esta funcíon es la encargada de encontrar las caras de una imagen utilizando los clasificadores en cascada
    y a continuación suprimiendo las caras reduntantes.
    '''
    fil,col = imagen.shape
    imagen = np.float64(imagen)
    imagen = preprocessing.scale(imagen.reshape(-1,1)).reshape(fil,col)#Normalización de la imagen

    imagen_integral = integral_image(imagen)
    prob_cara = np.ones(shape=(imagen.shape[0] - 23, imagen.shape[1] - 23, 2))

    for i in range(0,prob_cara.shape[0]):
        for j in range(0,prob_cara.shape[1]):
            #Sistema de detección en cascada con 5 clasificadores
            k=0
            while ( k < 5) and (prob_cara[i,j,0]==1):
                #Solo se calculan las caracteristicas de ese clasificador concreto
                caracteristicasVentana = haar_like_feature(imagen_integral,i,j,24,24,tipo_caracteristicas_validas[np.uint8(cord_validas[k])==1],caracteristicas_validas[np.uint8(cord_validas[k])==1])
                
                h = Clasificadores[k].predict_proba(caracteristicasVentana.reshape(1,-1)).ravel()
                # La probabilidad mínima para que una ventana sea cara hemos decido ponerla en 0.85
                if (h[1]>=0.85):
                    prob_cara[i,j,0] = 1
                    prob_cara[i,j,1] = h[1]
                else:
                    prob_cara[i,j,0] = 0
                    prob_cara[i,j,1] = 0
                k+=1
    if (np.sum(prob_cara[:,:,0]==1)>0):
        prob_cara = eliminar_caras_reduntantes(prob_cara,fil,col)
        caras =  np.argwhere(prob_cara[:,:,0]==1)
    else:
        caras = np.array([-1,-1])
    return caras

In [14]:
def anadirCarasAImagen(Imagen,caras,i):
    '''
    Esta función es la encargada de dibujar los rectangulos en la imagen original
    '''
    caras = np.float64(caras)
    caras = np.int64(caras * i)
    for x,y in caras:
        cv2.rectangle(Imagen,(x,y),(x + int(23*i),y + int(23*i)),(255,0,0),1)
    return Imagen

In [15]:
def main(imagen_original,Clasificadores,cord_validas_cascada,tipo_caracteristicas_validas,caracteristicas_validas):
    '''
    Esta función recibe la imagen original, los clasificadores, las caracteristicas importantes...
    y devulve la imagen con las caras rodeadas por un rectangulo.
    '''
    fil,col = imagen_original.shape
    imagen = np.copy(imagen_original)

    caras = detectar_caras(imagen,Clasificadores,cord_validas_cascada,tipo_caracteristicas_validas,caracteristicas_validas)
    caras = caras.reshape(-1,2)
    if (caras[0,0]!=-1):
        imagen_original = anadirCarasAImagen(imagen_original,caras,1)

    filIm,colIm = imagen_original.shape

    for i in range(2,5):#Reducir tres veces la imagen para detectar caras
        if ((filIm/i >= 24) and (colIm/i >= 24)):
            imagen = cv2.resize(imagen_original, (filIm/i, colIm/i), interpolation = cv2.INTER_AREA)
            caras = detectar_caras(imagen,Clasificadores,cord_validas_cascada,tipo_caracteristicas_validas,caracteristicas_validas)
            caras = caras.reshape(-1,2)
            if (caras[0,0]!=-1):
                imagen_original = anadirCarasAImagen(imagen_original,caras,i)

    for i in range(2,3):#Ampliar una vez la imagen para detectar caras ( Se podría ampliar mas veces pero es muy lento)
        imagen = cv2.resize(imagen_original, (filIm*i, colIm*i), interpolation = cv2.INTER_LINEAR)
        caras = detectar_caras(imagen,Clasificadores,cord_validas_cascada,tipo_caracteristicas_validas,caracteristicas_validas)
        caras = caras.reshape(-1,2)
        if (caras[0,0]!=-1):
            imagen_original = anadirCarasAImagen(imagen_original,caras,float(1./i))
            
    return imagen_original

In [24]:
#Preprocesamiento, obtención de caracteristicas y entrenamiento de clasificadores
print 'Prepocesando imagenes...'
matrizCaras,matrizNoCaras,filImagen,colImagen = preprocesamiento(1)#1 = dataset1
#matrizCaras,matrizNoCaras,filImagen,colImagen = preprocesamiento(2) 2 = dataset2
#NOTA: la ejecución del codigo es muy lenta con el dataset 2, recomendamos entrenar con el dataset 1
print 'Imagenes preprocesadas.'
print 'Obteniendo caracteristicas de Haar...'
cord,typecord,caracteristicasCaras,caracteristicasNoCaras = obtenerCaracteristicas(matrizCaras,matrizNoCaras,24,24)
print 'Caracteristicas de Haar de las imagenes de entreneamiento obtenidas.'
start = time()
print 'Entrenando clasificadores...'
Clasificadores,cord_validas_cascada,caracteristicas_validas,tipo_caracteristicas_validas = entrenarClasificadores(caracteristicasCaras,caracteristicasNoCaras,matrizCaras,matrizNoCaras,cord,typecord,60)
print 'Clasificadores entrenados en {} segundos '.format(time() - start)

Prepocesando imagenes...
Imagenes preprocesadas.
Obteniendo caracteristicas de Haar...
Caracteristicas de Haar de las imagenes de entreneamiento obtenidas.
Entrenando clasificadores...
Precisión del clasificador con 60 caracteristicas: 0.99
Precisión del clasificador con 5 caracteristicas: 0.98
Precisión del clasificador con 18 caracteristicas: 0.98
Precisión del clasificador con 32 caracteristicas: 0.98
Precisión del clasificador con 46 caracteristicas: 0.98
Clasificadores entrenados en 723.031000137 segundos 


In [59]:
#imagen_original = cv2.imread('mona_lisa.jpg',0)
imagen_original = cv2.imread('lena.png',0)

cv2.imshow('original',imagen_original)
cv2.waitKey(0)
cv2.destroyAllWindows()

print 'Detectando caras...'
imagen_con_caras = main(imagen_original,Clasificadores,cord_validas_cascada,tipo_caracteristicas_validas,caracteristicas_validas)
print 'Caras detectadas'

cv2.imshow('imagen con caras detectadas',imagen_con_caras)
cv2.waitKey(0)
cv2.destroyAllWindows()

Detectando caras...
Caras detectadas
